In [1]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import sklearn as sk
from sklearn.metrics import confusion_matrix

In [2]:
class confusionMatrix:
    def __init__(self, real, predicted, labels):
        
        self.real = real
        self.predicted = predicted
        self.labels = labels # = k
        self.mainArray = confusion_matrix(self.real, self.predicted, self.labels)
        
        self.hits = 0
        self.n = 0
        confusionMatrix.calcParameters(self)
        
        self.accuracy = 0
        confusionMatrix.accuracy(self)
    
    def calcParameters(self):
        
        for i in range(0, len(self.mainArray)):
            self.hits += self.mainArray[i,i]
            for j in range(0, len(self.mainArray)):
                self.n += self.mainArray[i,j]
                
    def accuracy(self):
        self.accuracy = self.hits/self.n
        
    def draw(self):
        print('to do')
        
        
    
ccc = confusionMatrix([1,2,4,3,3,3,4,0,1,2,4,3,1,4,2,2,3,0,0,0,4,4,2,4,2,4,1,3],
                      [0,1,3,4,0,2,3,4,2,2,3,4,1,3,2,4,2,3,0,1,1,2,3,4,4,2,0,3],
                      [0,1,2,3,4])
print(ccc.mainArray)
print(ccc.hits)
print(ccc.accuracy)
ccc.draw()


[[1 1 0 1 1]
 [2 1 1 0 0]
 [0 1 2 1 2]
 [1 0 2 1 2]
 [0 1 2 4 1]]
6
0.21428571428571427
to do
